In [1]:
from calculate_scores import *

total cores: 256
1 Physical GPUs, 1 Logical GPUs


2024-03-13 12:24:07.442509: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2024-03-13 12:24:07.442743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21321 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [2]:
bands=[29]
patch_size = 128
last_filter = 128
strides = 32    #patch_size
dates, times, labeled_data, x_cao, masks_cao, lon_lats_cao , max_vals, min_vals  = import_label_data("/uio/hume/student-u37/fslippe/data/labeled_data/results_backup_20240221")  
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

patches_cao, all_lon_patches_cao, all_lat_patches_cao, starts_cao, ends_cao, shapes_cao, n_patches_tot_cao, indices_cao = generate_patches([x[:,:,0] for x in x_cao],
                                                                                                                                            masks_cao,
                                                                                                                                            lon_lats_cao,
                                                                                                                                            max_vals,
                                                                                                                                            min_vals,
                                                                                                                                            autoencoder_predict,
                                                                                                                                            strides=[1, strides, strides,1])



In [ ]:
dates, times, labeled_data, x_cao, masks_cao, lon_lats_cao , max_vals, min_vals  = import_label_data("/uio/hume/student-u37/fslippe/data/labeled_data/results_backup_20240221")  



In [3]:
labeled_data['created_at'] = pd.to_datetime(labeled_data['created_at'])

# Define the cutoff time
cutoff_time = pd.Timestamp('2024-02-01', tz='UTC')  # Specify UTC timezone


# Split the DataFrame
before_cutoff = labeled_data[labeled_data['created_at'] < cutoff_time]
after_cutoff = labeled_data[labeled_data['created_at'] >= cutoff_time]

# Convert DataFrames back to JSON format
before_cutoff_json = before_cutoff.to_json(orient='records')
after_cutoff_json = after_cutoff.to_json(orient='records')


# Save the JSON files
with open("/uio/hume/student-u37/fslippe/data/labeled_data/lavterskel.json", 'w') as f_before:
    f_before.write(before_cutoff_json)

with open("/uio/hume/student-u37/fslippe/data/labeled_data/expert_group.json", 'w') as f_after:
    f_after.write(after_cutoff_json)


In [20]:
last_filter = 128
with tf.device('/CPU:0'):   
    encoded_patches_flat_cao = load_and_predict_encoder(patch_size, last_filter, patches_cao)


In [22]:
import functions 
importlib.reload(functions)
from functions import * 
import calculate_scores 
importlib.reload(calculate_scores)
from calculate_scores import * 
size_threshold = None#0
label_source = "only_lavterskel"
#label_source = "only_experts"

if label_source == "only_lavterskel":
    labeled_data = before_cutoff
elif label_source == "only_experts":
    labeled_data = after_cutoff

index_list = range(len(x_cao))
for n_K in [10, 11, 12, 13, 14, 15, 16]:#, 11, 12, 13, 14, 15, 16]:
    open_label = np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_opencell_label.npy")
    closed_label = np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_closedcell_label.npy")
    
    print("open", open_label)
    print("closed", closed_label)

    if not os.path.exists(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_opencell_label.npy"):
        manually_find_cloud_labels(min_vals, max_vals, autoencoder_predict, patch_size, last_filter, n_K)
    if not os.path.exists(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_closedcell_label.npy"):
        manually_find_cloud_labels(min_vals, max_vals, autoencoder_predict, patch_size, last_filter, n_K)

    open_label = np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_opencell_label.npy")
    closed_label = np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_closedcell_label.npy")
    
    
    labels, global_min, global_max = get_cluster_results(encoded_patches_flat_cao, patch_size, last_filter, n_K)
    print(global_max)
    
    label_map, lon_map, lat_map = process_label_maps(labels,
                                                    all_lon_patches_cao,
                                                    all_lat_patches_cao,
                                                    starts_cao,                 
                                                    ends_cao,
                                                    shapes_cao,         
                                                    indices_cao,
                                                    global_max,             
                                                    n_patches_tot_cao,              
                                                    patch_size,
                                                    strides,
                                                    closed_label, 
                                                    open_label, 
                                                    size_thr_1=size_threshold, 
                                                    size_thr_2=size_threshold)

        # Example usage of the function
    extent = [-15, 25, 58, 84]
    # plot_filtered_map(label_map, lon_map, lat_map, idx, extent, global_max, dates)
    # plt.show()
    valid_lons = []
    valid_lats = []

    for i in index_list:
        valid_lon, valid_lat = get_valid_lons_lats(x_cao[i][:,:,0],
                                                    lon_lats_cao[i],
                                                    label_map[i],
                                                    lon_map[i],
                                                    lat_map[i],
                                                    dates[i],
                                                    times[i],
                                                    open_label=open_label,
                                                    closed_label=closed_label,
                                                    p_level=950,
                                                    angle_thr=5,
                                                    size_threshold_1=None,
                                                    size_threshold_2=None,
                                                    plot=False,
                                                    extent= [-15, 25, 58, 84])
        valid_lons.append(valid_lon)
        valid_lats.append(valid_lat)                                            


    model_borders, model_areas = process_model_masks(index_list, lon_map, lat_map, valid_lons, valid_lats, indices_cao, label_map, closed_label, open_label, brush=False, plot=False)

    labeled_areas, labeled_borders = get_area_and_border_mask(x_cao, dates, times, masks_cao, labeled_data, reduction=strides, index_list=index_list, plot=False)



    # cmap_tab10 = plt.cm.tab10
    # cmap_tab20 = plt.cm.tab20
    # colors_tab20 = cmap_tab20(np.arange(cmap_tab20.N))[1::2]
    # colors_tab10 = cmap_tab10(np.arange(cmap_tab10.N))
    # extra_colors = colors_tab20
    # black = np.array([0, 0, 0, 1])
    # colors_new = np.vstack((colors_tab10, colors_tab20))[:np.max(global_max)-1]
    # colors_new = np.vstack((colors_new, black))
    # label_cmap = mcolors.ListedColormap(colors_new)

    # for i in index_list:
    #     plot_areas_w_scores(x_cao[i], label_map[i], label_cmap, labeled_areas[i], model_areas[i], area_scores[i], weighted_area_scores[i])
    # fig, axs = plt.subplots(1,2, figsize=[10,10])
    # cb = axs[0].imshow(labeled_areas[0])
    # plt.colorbar(cb)
    # cb = axs[1].imshow(labeled_borders[0])
    # plt.colorbar(cb)


    all_area_scores, all_border_scores= calculate_scores_and_plot(model_borders, model_areas, labeled_borders, labeled_areas, plot=False)

    
    folder = f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{last_filter}/clustering/scores/"
    if not os.path.exists(folder):
        os.makedirs(folder)

    np.save(folder + f"area_scores_cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_res{strides}_thr{size_threshold}_{label_source}", all_area_scores)
    np.save(folder + f"border_scores_cluster_dnb_l95_z50_ps{patch_size}_band29_filter{last_filter}_K{n_K}_res{strides}_thr{size_threshold}_{label_source}", all_border_scores)
    

1 Physical GPUs, 1 Logical GPUs
open 4
closed 7
cluster load loc: /uio/hume/student-u37/fslippe/data/models/patch_size128/filter128/clustering/cluster_dnb_l95_z50_ps128_band29_filter128_K10.pkl
10


/uio/hume/student-u37/fslippe/master_project/code/functions.py:348: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pat_lon = np.concatenate(np.array(pat_lon), axis=0)
/uio/hume/student-u37/fslippe/master_project/code/functions.py:349: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pat_lat = np.concatenate(np.array(pat_lat), axis=0)


178.22037
180.09615624544423
nan
nan
nan
nan


/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


245.34851
195.30852327717403
nan
nan
nan
nan
nan
nan


...

/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


163.73222
216.95978091845504
210.36807
216.96859123737966
8.271177
160.65448310671115
nan
nan


/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


142.744
196.30365054399616
58.055557
187.65130397515657
81.65582
217.60334517234622
168.51204
211.22279789400415
nan
nan


/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/software/easybuild/software/SciPy-bundle/2021.10-foss-2021b/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/uio/hume/student-u37/fslippe/master_project/code/functions.py:673: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  border_lines = np.array(di["data.borderLines"], dtype=object)
/uio/hume/student-u37/fslippe/master_project/code/functions.py:482: RuntimeWarning: divide by zero encountered in true_divide
  border_weighted_frac = np.where(m_preds, ab

In [ ]:
print(all_area_scores.keys())

for key, item in all_area_scores.items():
    print(key, np.mean(item)/ np.mean(all_area_scores["tot_points"]))

In [ ]:
cmap_tab10 = plt.cm.tab10
cmap_tab20 = plt.cm.tab20
colors_tab20 = cmap_tab20(np.arange(cmap_tab20.N))[1::2]
colors_tab10 = cmap_tab10(np.arange(cmap_tab10.N))
extra_colors = colors_tab20
black = np.array([0, 0, 0, 1])
colors_new = np.vstack((colors_tab10, colors_tab20))[:np.max(global_max)-1]
colors_new = np.vstack((colors_new, black))
label_cmap = mcolors.ListedColormap(colors_new)


def plot_areas_w_scores(x_cao, label_map, label_cmap, labeled_area, model_area, score, weighted_score):
    fig, axs = plt.subplots(1, 4, figsize=[20,10])
    fig.suptitle("Score: %.4f    Weighted score %.4f" %(score, weighted_score))

    cb = axs[0].imshow(x_cao, cmap="gray_r")
    plt.colorbar(cb)

    cb = axs[1].imshow(label_map, cmap=label_cmap)
    plt.colorbar(cb)

    axs[2].set_title("Labeled area")
    cb = axs[2].imshow(labeled_area, vmin=0, vmax=1)
    plt.colorbar(cb)

    axs[3].set_title("Model area")
    cb = axs[3].imshow(model_area, vmin=0, vmax=1)
    plt.colorbar(cb)
    
    plt.show()

def plot_borders_w_scores(x_cao, label_map, label_cmap, labeled_border, model_border, score, weighted_score):
    fig, axs = plt.subplots(1, 4, figsize=[20,10])
    fig.suptitle("Score: %.4f    Weighted score %.4f" %(score, weighted_score))

    cb = axs[0].imshow(x_cao, cmap="gray_r")
    plt.colorbar(cb)

    cb = axs[1].imshow(label_map, cmap=label_cmap)
    plt.colorbar(cb)

    axs[2].set_title("Labeled border")
    cb = axs[2].imshow(labeled_border, vmin=0, vmax=1)
    plt.colorbar(cb)

    axs[3].set_title("Model border")
    cb = axs[3].imshow(model_border, vmin=0, vmax=1)
    plt.colorbar(cb)

    plt.show()

for i in index_list:
    plot_areas_w_scores(x_cao[i], label_map[i], label_cmap, labeled_areas[i], model_areas[i], area_scores[i], weighted_area_scores[i])
    #plot_borders_w_scores(x_cao[i], label_map[i], label_cmap, labeled_borders[i], model_borders[i], border_scores[i], weighted_border_scores[i])




In [ ]:
cmap_tab10 = plt.cm.tab10
cmap_tab20 = plt.cm.tab20
colors_tab20 = cmap_tab20(np.arange(cmap_tab20.N))[1::2]
colors_tab10 = cmap_tab10(np.arange(cmap_tab10.N))
extra_colors = colors_tab20
black = np.array([0, 0, 0, 1])
colors_new = np.vstack((colors_tab10, colors_tab20))[:np.max(global_max)-1]
colors_new = np.vstack((colors_new, black))

new_cmap = mcolors.ListedColormap(colors_new)
label_map, lon_map, lat_map = process_label_maps(labels,
                                                    all_lon_patches_cao,
                                                    all_lat_patches_cao,
                                                    starts_cao,
                                                    ends_cao,
                                                    shapes_cao,
                                                    indices_cao,
                                                    global_max,
                                                    n_patches_tot_cao,
                                                    patch_size,
                                                    strides,
                                                    closed_label, 
                                                    open_label, 
                                                    size_thr_1=0, 
                                                    size_thr_2=0)
for i in range(0, 30):
    print(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{last_filter}_K{n_K}_opencell_label.npy"))
    print(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{last_filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{last_filter}_K{n_K}_closedcell_label.npy"))

    fig, axs = plt.subplots(1, 2, figsize=[10,10])
    axs[0].imshow(x_cao[i], cmap="gray_r")
    cb = axs[1].imshow(label_map[i], cmap=new_cmap)
    plt.colorbar(cb)
    plt.show()

In [ ]:
index_list = [10]

import functions 
importlib.reload(functions)
from functions import *
# label_map, lon_map, lat_map = process_label_maps(labels,
#                                                     all_lon_patches_cao,
#                                                     all_lat_patches_cao,
#                                                     starts_cao,
#                                                     ends_cao,
#                                                     shapes_cao,
#                                                     indices_cao,
#                                                     global_max,
#                                                     n_patches_tot_cao,
#                                                     patch_size,
#                                                     strides,
#                                                     closed_label, 
#                                                     open_label, 
#                                                     size_thr_1=20, 
#                                                     size_thr_2=20)

    # Example usage of the function
extent = [-15, 25, 58, 84]
# plot_filtered_map(label_map, lon_map, lat_map, idx, extent, global_max, dates)
# plt.show()
# valid_lons = np.empty(index_list[-1]+1, dtype=object)
# valid_lats = np.empty(index_list[-1]+1, dtype=object)

# for i in index_list:
#     plt.imshow(masks_cao[i])
#     plt.show()
#     plt.imshow(label_map[i], cmap="tab10")
#     plt.colorbar()
#     plt.show()
#     plt.imshow(x_cao[i], cmap="gray_r")
#     plt.show()
#     valid_lon, valid_lat = get_valid_lons_lats(x_cao[i][:,:,0],
#                                                 lon_lats_cao[i],
#                                                 label_map[i],
#                                                 lon_map[i],
#                                                 lat_map[i],
#                                                 dates[i],
#                                                 times[i],
#                                                 open_label=closed_label,
#                                                 closed_label=open_label,
#                                                 p_level=950,
#                                                 angle_thr=5,
#                                                 size_threshold_1=None,
#                                                 size_threshold_2=None,
#                                                 plot=True,
#                                                 extent= [-15, 25, 58, 84])
#     valid_lons[i] = (valid_lon)
#     valid_lats[i] = (valid_lat)

model_boundaries, model_areas = process_model_masks(index_list, lon_map, lat_map, valid_lons, valid_lats, indices_cao, label_map, closed_label, open_label, brush=False, plot=True)
#model_borders, model_areas = process_model_masks(index_list, lon_map, lat_map, valid_lons, valid_lats, indices_cao, label_map, closed_label, open_label, brush=False, plot=False)

dates_idx = [dates[idx] for idx in index_list]
times_idx = [times[idx] for idx in index_list]

#labeled_areas, labeled_boundaries = get_area_and_border_mask(x_cao, dates, times, masks_cao, labeled_data, reduction=strides, index_list=index_list, plot=True)
# fig, axs = plt.subplots(1,2, figsize=[10,10])
# cb = axs[0].imshow(labeled_areas[0])
# plt.colorbar(cb)
# cb = axs[1].imshow(labeled_boundaries[0])
# plt.colorbar(cb)


#area_scores, border_scores, weighted_area_score, weighted_border_score = calculate_scores_and_plot(model_boundaries, model_areas, labeled_boundaries, labeled_areas, plot=True)

In [ ]:
bands=[29]
patch_size = 128
last_filter = 128
strides = 32    #patch_size
dates, times, labeled_data, x_cao, masks_cao, lon_lats_cao , max_vals, min_vals  = import_label_data("/uio/hume/student-u37/fslippe/data/labeled_data/results_backup_20240221")  
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

patches_cao, all_lon_patches_cao, all_lat_patches_cao, starts_cao, ends_cao, shapes_cao, n_patches_tot_cao, indices_cao = generate_patches([x[:,:,0] for x in x_cao],
                                                                                                                                            masks_cao,
                                                                                                                                            lon_lats_cao,
                                                                                                                                            max_vals,
                                                                                                                                            min_vals,
                                                                                                                                            autoencoder_predict,
                                                                                                                                            strides=[1, strides, strides,1])



In [ ]:
labeled_data['created_at'] = pd.to_datetime(labeled_data['created_at'])

# Define the cutoff time
cutoff_time = pd.Timestamp('2024-02-01', tz='UTC')  # Specify UTC timezone


# Split the DataFrame
before_cutoff = labeled_data[labeled_data['created_at'] < cutoff_time]
after_cutoff = labeled_data[labeled_data['created_at'] >= cutoff_time]

# Convert DataFrames back to JSON format
before_cutoff_json = before_cutoff.to_json(orient='records')
after_cutoff_json = after_cutoff.to_json(orient='records')


# Save the JSON files
with open("/uio/hume/student-u37/fslippe/data/labeled_data/lavterskel.json", 'w') as f_before:
    f_before.write(before_cutoff_json)

with open("/uio/hume/student-u37/fslippe/data/labeled_data/expert_group.json", 'w') as f_after:
    f_after.write(after_cutoff_json)


In [ ]:
bands=[29]
patch_size = 128
last_filter = 128
strides = 32    #patch_size
dates, times, labeled_data, x_cao, masks_cao, lon_lats_cao , max_vals, min_vals  = import_label_data("/uio/hume/student-u37/fslippe/data/labeled_data/results_backup_20240221")  
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

patches_cao, all_lon_patches_cao, all_lat_patches_cao, starts_cao, ends_cao, shapes_cao, n_patches_tot_cao, indices_cao = generate_patches([x[:,:,0] for x in x_cao],
                                                                                                                                            masks_cao,
                                                                                                                                            lon_lats_cao,
                                                                                                                                            max_vals,
                                                                                                                                            min_vals,
                                                                                                                                            autoencoder_predict,
                                                                                                                                            strides=[1, strides, strides,1])



In [ ]:
labeled_data['created_at'] = pd.to_datetime(labeled_data['created_at'])

# Define the cutoff time
cutoff_time = pd.Timestamp('2024-02-01', tz='UTC')  # Specify UTC timezone


# Split the DataFrame
before_cutoff = labeled_data[labeled_data['created_at'] < cutoff_time]
after_cutoff = labeled_data[labeled_data['created_at'] >= cutoff_time]

# Convert DataFrames back to JSON format
before_cutoff_json = before_cutoff.to_json(orient='records')
after_cutoff_json = after_cutoff.to_json(orient='records')


# Save the JSON files
with open("/uio/hume/student-u37/fslippe/data/labeled_data/lavterskel.json", 'w') as f_before:
    f_before.write(before_cutoff_json)

with open("/uio/hume/student-u37/fslippe/data/labeled_data/expert_group.json", 'w') as f_after:
    f_after.write(after_cutoff_json)


In [ ]:
import functions 
importlib.reload(functions)
from functions import *
labeled_areas, labeled_boundaries = get_area_and_border_mask(x_cao, dates, times, masks_cao, labeled_data, reduction=strides, index_list=index_list, plot=True)
